In [130]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [131]:
df = pd.read_csv('./2차전처리데이터.csv',encoding='cp949')

---

# anova 결과에 따른  <요일>, <피해자성별>, <교차로형태>, <노인보호구역_여부>,    <어린이보호구역_여부>컬럼 삭제

In [132]:
df = df.drop(['요일','피해자성별','노인보호구역_여부','어린이보호구역_여부'],axis=1)

---

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57570 entries, 0 to 57569
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   주야          57570 non-null  object
 1   사고내용        57570 non-null  object
 2   사고유형        57570 non-null  object
 3   가해자성별       57570 non-null  object
 4   연령가해자       57570 non-null  object
 5   가해자음주       57570 non-null  object
 6   법규위반가해자     57570 non-null  object
 7   행동유형가해자     57570 non-null  object
 8   신체상해정도가해자   57570 non-null  object
 9   신체상해정도피해자   57570 non-null  object
 10  가해자신체상해주부위  57570 non-null  object
 11  피해자신체상해주부위  57570 non-null  object
 12  당사자종별가해자    57570 non-null  object
 13  차량용도가해자     57570 non-null  object
 14  도로종류        57570 non-null  object
 15  도로형태        57570 non-null  object
 16  도로선형        57570 non-null  object
 17  기상상태        57570 non-null  object
 18  노면상태        57570 non-null  object
 19  교차로형태       57570 non-null  object
 20  사고심각도 

# 가변수화 또는 순석척도로 변경하기 전 각 컬럼의 속성의 개수 줄이기

## 1. 연령가해자

### 20세이하 21-30세   18.852  0.001 14.2807 23.4233   True
### 20세이하 31-40세  19.2903  0.001 15.0615 23.5191   True
### 20세이하 41-50세  19.8305  0.001 15.7742 23.8869   True
### 20세이하 51-60세  20.2524  0.001  16.267 24.2379   True
### 20세이하 61-64세  20.8893  0.001 16.2525 25.5261   True
### 20세이하  65세이상  20.0477  0.001 15.7783  24.317   True

======================================================================
###  20세 이하와 나머지 연령 사이에서 큰 차이가 있음
###  특히 20세 이하와 61-64세에서 큰 차이를 보이고 있음
###  20세이하 - 아동청소년 21-30세 청년 / 31세 - 50세 중년 / 51세 - 64세 장년 / 65세 이상 노인 으로 변경
###   
### 그러나 20세 이하가 사고 심각도에 미치는 영향은 미비 
======================================================================

In [134]:
idx = df[df['연령가해자']=='20세이하'].index
for i in idx:
    df.iloc[:,4][i]=df.iloc[:,4][i].replace('20세이하','아동청소년')
    
idx = df[df['연령가해자']=='21-30세'].index
for i in idx:
    df.iloc[:,4][i]=df.iloc[:,4][i].replace('21-30세','청년')
    
idx = df[df['연령가해자']=='31-40세'].index
for i in idx:
    df.iloc[:,4][i]=df.iloc[:,4][i].replace('31-40세','중년')
    
idx = df[df['연령가해자']=='41-50세'].index
for i in idx:
    df.iloc[:,4][i]=df.iloc[:,4][i].replace('41-50세','중년')
    
idx = df[df['연령가해자']=='51-60세'].index
for i in idx:
    df.iloc[:,4][i]=df.iloc[:,4][i].replace('51-60세','장년')
    
idx = df[df['연령가해자']=='61-64세'].index
for i in idx:
    df.iloc[:,4][i]=df.iloc[:,4][i].replace('61-64세','장년')
    
idx = df[df['연령가해자']=='65세이상'].index
for i in idx:
    df.iloc[:,4][i]=df.iloc[:,4][i].replace('65세이상','노인')

---

## 2. 법규위반가해자

### 과속이 제일 큰 영향을 끼치고 있음
### 안전거리 미확보, 안전운전 의무 불이행 를 묶어서 '안전운전 의무 불이행'으로 처리
### 보행자 보호의무 위반, 신호 위반을 묶어서 ' 신호 또는 보행자 보호 위반'으로 처리

In [135]:
idx = df[df['법규위반가해자']=='안전거리 미확보'].index
for i in idx:
    df.iloc[:,6][i]=df.iloc[:,6][i].replace('안전거리 미확보','안전운전 의무 불이행')

idx = df[df['법규위반가해자']=='보행자 보호의무 위반'].index
for i in idx:
    df.iloc[:,6][i]=df.iloc[:,6][i].replace('보행자 보호의무 위반','신호 또는 보행자보호 위반')
    
idx = df[df['법규위반가해자']=='신호위반'].index
for i in idx:
    df.iloc[:,6][i]=df.iloc[:,6][i].replace('신호위반','신호 또는 보행자보호 위반')
    

In [136]:
df['법규위반가해자'].unique()

array(['안전운전 의무 불이행', '신호 또는 보행자보호 위반', '기타', '중앙선 침범', '교차로 통행방법 위반',
       '과속'], dtype=object)

---

## 3. 행동유형가해자

### 후진 중, 출발 중, 좌우회전 중, U턴 중, 앞지르기 중 => 주행 중 
### 주행 중 대기, 주정차중 => 주정차 중 
### 진로변경 중, 직진 중 => 직진 또는 진로변경 중

In [137]:
idx = df[(df['행동유형가해자']=='후진 중')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('후진 중','주행 중')

idx = df[(df['행동유형가해자']=='출발 중')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('출발 중','주행 중')
    
idx = df[(df['행동유형가해자']=='좌우회전 중')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('좌우회전 중','주행 중')
    
idx = df[(df['행동유형가해자']=='U턴 중')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('U턴 중','주행 중')
    
idx = df[(df['행동유형가해자']=='앞지르기 중')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('앞지르기 중','주행 중')

In [138]:
idx = df[(df['행동유형가해자']=='주행 중 대기')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('주행 중 대기','주정차 중')

idx = df[(df['행동유형가해자']=='주정차중')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('주정차중','주정차 중')

In [139]:
idx = df[(df['행동유형가해자']=='진로변경 중')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('진로변경 중','직진 또는 진로변경 중')

idx = df[(df['행동유형가해자']=='직진 중')].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('직진 중','직진 또는 진로변경 중')

In [140]:
df['행동유형가해자'].unique()

array(['직진 또는 진로변경 중', '주행 중', '기타/불명', '주정차 중'], dtype=object)

---

## 가해자신체상해주부위
####  
### 큰 차이가 두드러지지 않아 대분류로 묶겠음

### ===============================================
### 가슴   상해없음  -46.3251  0.001   -81.479 -11.1712   True
### 가슴      팔  -51.3611 0.0486  -102.586  -0.1362   True
### 대퇴부     뒷목   89.0109 0.0301    3.7219 174.2999   True
### 뒷목      발   -90.607 0.0158 -173.3356  -7.8785   True
### 뒷목   상해없음  -79.7049 0.0112 -150.7859  -8.6239   True
### 뒷목      팔  -84.7409 0.0259 -164.9949  -4.4869   True
### 뒷목    하퇴부  -78.5909 0.0443 -156.3328   -0.849   True
### ===============================================

### 상해없음, 없음 => 상해없음
### 기타, 불명 => 기타/불명
###  얼굴, 목,뒷목,머리,앞목 => 머리/얼굴/목
###  가슴, 팔, 허리,어깨,손,등 => 상반신
###  발,하퇴부,대퇴부 => 하반신

In [141]:
idx = df[(df['가해자신체상해주부위']=='가슴')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('가슴','상반신')
    
idx = df[(df['가해자신체상해주부위']=='팔')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('팔','상반신')
    
idx = df[(df['가해자신체상해주부위']=='허리')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('허리','상반신')
    
idx = df[(df['가해자신체상해주부위']=='어깨')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('어깨','상반신')
    
idx = df[(df['가해자신체상해주부위']=='손')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('손','상반신')
    
idx = df[(df['가해자신체상해주부위']=='등')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('등','상반신')

In [142]:
idx = df[(df['가해자신체상해주부위']=='얼굴')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('얼굴','머리/얼굴/목')
    
idx = df[(df['가해자신체상해주부위']=='목')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('목','머리/얼굴/목')
    
idx = df[(df['가해자신체상해주부위']=='뒷목')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('뒷목','머리/얼굴/목')
    
idx = df[(df['가해자신체상해주부위']=='머리')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('머리','머리/얼굴/목')
    
idx = df[(df['가해자신체상해주부위']=='앞목')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('앞목','머리/얼굴/목')


In [143]:
idx = df[(df['가해자신체상해주부위']=='기타')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('기타','기타/불명')
    
idx = df[(df['가해자신체상해주부위']=='불명')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('불명','기타/불명')

In [144]:
idx = df[(df['가해자신체상해주부위']=='없음')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('없음','상해없음')

In [145]:
idx = df[(df['가해자신체상해주부위']=='발')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('발','하반신')
    
idx = df[(df['가해자신체상해주부위']=='하퇴부')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('하퇴부','하반신')

idx = df[(df['가해자신체상해주부위']=='대퇴부')].index
for i in idx:
    df.iloc[:,10][i]=df.iloc[:,10][i].replace('대퇴부','하반신')    

In [146]:
df['가해자신체상해주부위'].unique()

array(['상해없음', '기타/불명', '머리/얼굴/목', '상반신', '하반신'], dtype=object)

---

## 피해자신체상해주부위
###    
### 가해자신체상해주부위와 동일하게 묶겠음

In [147]:
idx = df[(df['피해자신체상해주부위']=='하퇴부')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('하퇴부','하반신')

idx = df[(df['피해자신체상해주부위']=='발')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('발','하반신')
    
idx = df[(df['피해자신체상해주부위']=='대퇴부')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('대퇴부','하반신')
    

idx = df[(df['피해자신체상해주부위']=='머리')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('머리','머리/얼굴/목')
    
idx = df[(df['피해자신체상해주부위']=='목')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('목','머리/얼굴/목')
    
idx = df[(df['피해자신체상해주부위']=='얼굴')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('얼굴','머리/얼굴/목')
    
idx = df[(df['피해자신체상해주부위']=='뒷목')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('뒷목','머리/얼굴/목')
    
idx = df[(df['피해자신체상해주부위']=='앞목')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('앞목','머리/얼굴/목')
    
    
idx = df[(df['피해자신체상해주부위']=='허리')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('허리','상반신')
    
idx = df[(df['피해자신체상해주부위']=='팔')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('팔','상반신')
    
idx = df[(df['피해자신체상해주부위']=='손')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('손','상반신')
    
idx = df[(df['피해자신체상해주부위']=='가슴')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('가슴','상반신')

idx = df[(df['피해자신체상해주부위']=='배')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('배','상반신')
    
idx = df[(df['피해자신체상해주부위']=='어깨')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('어깨','상반신')
    
idx = df[(df['피해자신체상해주부위']=='등')].index
for i in idx:
    df.iloc[:,11][i]=df.iloc[:,11][i].replace('등','상반신')

---

## 당사자종별가해자

### 원동기장치자전거, 이륜차, 자전거 => 원동기/이륜/자전거
### 승합차, 승용차, 경형, 중형  => 승합/승용차
### 특수차 > 화물차 => 특수/화물차
### 사발이 > 사륜오토바이 => 사발이/ATV### 건설기계  
### 기타, 불명 => 기타/불명
### 소형, 대형 

In [148]:
idx = df[(df['당사자종별가해자']=='원동기장치자전거')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('원동기장치자전거','원동기/이륜/자전거')
    
idx = df[(df['당사자종별가해자']=='이륜차')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('이륜차','원동기/이륜/자전거')
    
idx = df[(df['당사자종별가해자']=='자전거')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('자전거','원동기/이륜/자전거')
    
    
idx = df[(df['당사자종별가해자']=='기타')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('기타','기타/불명')

idx = df[(df['당사자종별가해자']=='불명')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('불명','기타/불명')
    
    
idx = df[(df['당사자종별가해자']=='사발이')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('사발이','사발이/ATV')

idx = df[(df['당사자종별가해자']=='사륜오토바이(ATV)')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('사륜오토바이(ATV)','사발이/ATV')
    
    
    
idx = df[(df['당사자종별가해자']=='특수차')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('특수차','특수/화물차')

idx = df[(df['당사자종별가해자']=='화물차')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('화물차','특수/화물차')

In [149]:
idx = df[(df['당사자종별가해자']=='승합차')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('승합차','승합/승용차')
      
    
idx = df[(df['당사자종별가해자']=='승용차')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('승용차','승합/승용차')
    
    
    
idx = df[(df['당사자종별가해자']=='소형')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('소형','소형/경형')
    
idx = df[(df['당사자종별가해자']=='경형')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('경형','소형/경형')
    
idx = df[(df['당사자종별가해자']=='대형')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('대형','중형/대형')
    
idx = df[(df['당사자종별가해자']=='중형')].index
for i in idx:
    df.iloc[:,12][i]=df.iloc[:,12][i].replace('중형','중형/대형')

---

## 차량용도가해자

### 기타, 불명, 비사업용기타 => 기타/불명
### 개별화물, 일반화물, 용달화물, 덤프트럭 => 화물차
### 마을버스,시내버스 => 시내/마을버스
### 고속버스,전세버스,버스, 시외버스 => 고속/시외/전세버스
### 개인택시, 법인택시 => 택시
### 기타/건설기계 => 건설기계
### 특수여객(장의), 위험물운송 => 특수여객/위험물운송

In [150]:
idx = df[(df['차량용도가해자']=='기타')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('기타','기타/불명')
    
idx = df[(df['차량용도가해자']=='불명')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('불명','기타/불명')
    
idx = df[(df['차량용도가해자']=='비사업용기타')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('비사업용기타','기타/불명')

In [151]:
idx = df[(df['차량용도가해자']=='개별화물')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('개별화물','화물차')
    
idx = df[(df['차량용도가해자']=='마을버스')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('마을버스','시내/마을버스')
    
    
idx = df[(df['차량용도가해자']=='고속버스')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('고속버스','고속/시외/전세버스')
    
idx = df[(df['차량용도가해자']=='전세버스')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('전세버스','고속/시외/전세버스')
    
idx = df[(df['차량용도가해자']=='버스')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('버스','고속/시외/전세버스')

In [152]:
idx = df[(df['차량용도가해자']=='일반화물')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('일반화물','화물차')
   
idx = df[(df['차량용도가해자']=='용달화물')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('용달화물','화물차')

In [153]:
idx = df[(df['차량용도가해자']=='개인택시')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('개인택시','택시')
    
idx = df[(df['차량용도가해자']=='법인택시')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('법인택시','택시')

In [154]:
idx = df[(df['차량용도가해자']=='시내버스')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('시내버스','시내/마을버스')

In [155]:
idx = df[(df['차량용도가해자']=='기타/건설기계')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('기타/건설기계','건설기계')

In [156]:
idx = df[(df['차량용도가해자']=='특수여객(장의)')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('특수여객(장의)','특수여객/위험물운송')
    
idx = df[(df['차량용도가해자']=='위험물운송')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('특수여객(장의)','특수여객/위험물운송')

In [157]:
idx = df[(df['차량용도가해자']=='덤프트럭')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('덤프트럭','화물차')

In [158]:
idx = df[(df['차량용도가해자']=='시외버스')].index
for i in idx:
    df.iloc[:,13][i]=df.iloc[:,13][i].replace('시외버스','고속/시외/전세버스')

---

## 도로종류

### 고속국도, 일반국도 => 일반/고속국도

In [159]:
idx = df[(df['도로종류']=='고속국도')].index
for i in idx:
    df.iloc[:,14][i]=df.iloc[:,14][i].replace('고속국도','일반/고속국도')
    
idx = df[(df['도로종류']=='일반국도')].index
for i in idx:
    df.iloc[:,14][i]=df.iloc[:,14][i].replace('일반국도','일반/고속국도')

---

## 도로형태

### 교차로내, 교차로부근, 교차로횡단보도내 => 교차로
### 횡단보도부근, 횡단보도상 => 횡단보도
### 터널안, 지하차도(도로)내 => 터널/지하차도

In [160]:
idx = df[(df['도로형태']=='교차로내')].index
for i in idx:
    df.iloc[:,15][i]=df.iloc[:,15][i].replace('교차로내','교차로')
    
idx = df[(df['도로종류']=='교차로부근')].index
for i in idx:
    df.iloc[:,15][i]=df.iloc[:,15][i].replace('교차로부근','교차로')
    
idx = df[(df['도로종류']=='교차로횡단보도내')].index
for i in idx:
    df.iloc[:,15][i]=df.iloc[:,15][i].replace('교차로횡단보도내','교차로')
    
    
    
idx = df[(df['도로형태']=='횡단보도부근')].index
for i in idx:
    df.iloc[:,15][i]=df.iloc[:,15][i].replace('횡단보도부근','횡단보도')
    
idx = df[(df['도로종류']=='횡단보도상')].index
for i in idx:
    df.iloc[:,15][i]=df.iloc[:,15][i].replace('횡단보도상','횡단보도')
    
    
idx = df[(df['도로형태']=='터널안')].index
for i in idx:
    df.iloc[:,15][i]=df.iloc[:,15][i].replace('터널안','터널/지하차도')
    
idx = df[(df['도로종류']=='지하차도(도로)내')].index
for i in idx:
    df.iloc[:,15][i]=df.iloc[:,15][i].replace('지하차도(도로)내','터널/지하차도')

---

## 도로선형

### 오르막, 내리막 => 경사로

In [161]:
idx = df[(df['도로선형']=='오르막')].index
for i in idx:
    df.iloc[:,16][i]=df.iloc[:,16][i].replace('오르막','경사로')
    
idx = df[(df['도로선형']=='내리막')].index
for i in idx:
    df.iloc[:,16][i]=df.iloc[:,16][i].replace('내리막','경사로')

In [162]:
df['도로선형'].unique()

array(['평지', '경사로', '기타/서비스구역'], dtype=object)

---

In [163]:
df

,주야,사고내용,사고유형,가해자성별,연령가해자,가해자음주,법규위반가해자,행동유형가해자,신체상해정도가해자,신체상해정도피해자,...,피해자신체상해주부위,당사자종별가해자,차량용도가해자,도로종류,도로형태,도로선형,기상상태,노면상태,교차로형태,사고심각도
0,주,경상,보도_길가장자리_기타,여,아동청소년,기타,안전운전 의무 불이행,직진 또는 진로변경 중,상해없음,경상,...,하반신,원동기/이륜/자전거,자전거,기타,기타단일로,평지,맑음,서리/결빙,교차로아님,2
1,주,중상,횡단중,남,노인,기타,안전운전 의무 불이행,직진 또는 진로변경 중,상해없음,중상,...,머리/얼굴/목,소형/경형,승용차,시도,기타단일로,평지,맑음,건조,교차로아님,32
2,야,경상,보도_길가장자리_기타,남,아동청소년,기타,안전운전 의무 불이행,직진 또는 진로변경 중,상해없음,경상,...,기타,소형/경형,승용차,시도,기타단일로,평지,맑음,건조,교차로아님,2
3,주,중상,보도_길가장자리_기타,남,청년,기타,안전운전 의무 불이행,주행 중,상해없음,중상,...,기타,중형/대형,화물차,기타,기타단일로,평지,맑음,건조,교차로아님,32
4,주,중상,횡단중,여,중년,기타,안전운전 의무 불이행,직진 또는 진로변경 중,상해없음,중상,...,머리/얼굴/목,중형/대형,승용차,기타,기타단일로,평지,맑음,건조,교차로아님,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57565,주,경상,보도_길가장자리_기타,여,장년,기타,안전운전 의무 불이행,직진 또는 진로변경 중,상해없음,경상,...,상반신,승합/승용차,승용차,시도,지하차도(도로)내,평지,맑음,건조,교차로아님,2
57566,주,경상,보도_길가장자리_기타,여,청년,해당 없음,기타,주행 중,상해없음,경상,...,상반신,승합/승용차,승용차,군도,교차로부근,평지,맑음,건조,교차로 - 삼지,2
57567,주,중상,차도통행중,여,중년,해당 없음,안전운전 의무 불이행,직진 또는 진로변경 중,상해없음,중상,...,하반신,승합/승용차,승용차,군도,지하차도(도로)내,평지,맑음,건조,교차로아님,32
57568,주,중상,보도_길가장자리_기타,남,노인,해당 없음,안전운전 의무 불이행,직진 또는 진로변경 중,상해없음,중상,...,하반신,승합/승용차,승용차,시도,기타단일로,평지,맑음,건조,교차로아님,32


In [164]:
df.to_csv('3차전처리데이터_분산분석.csv',encoding='cp949')